In [103]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd
import numpy as np
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import time
import random
import re
from sportsreference.ncaab.boxscore import Boxscore
from sportsreference.ncaab.roster import Player
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1993&year_max=2015&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=0'
r = requests.get(draft_url)

In [3]:
r.status_code

200

In [4]:
soup = BeautifulSoup(r.content, "html")

In [5]:
#print(soup.prettify())

In [6]:
div = soup.find("div", {"class": "table_outer_container"})
table = div.find("tbody")

In [7]:
table = div.find("tbody")
table

<tbody>
<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="right" data-stat="year_id">2003</td><td class="left" data-stat="lg_id"><a href="/draft/NBA_2003.html">NBA</a></td><td class="right" csk="1" data-stat="round">1</td><td class="right" csk="1" data-stat="pick_overall">1</td><td class="left" data-stat="team_id">CLE</td><td class="left" csk="James,LeBron" data-append-csv="jamesle01" data-stat="player"><a href="/players/j/jamesle01.html">LeBron James</a></td><td class="right" data-stat="age">18.182</td><td class="center" data-stat="pos">F-G</td><td class="right" data-stat="birth_country"><span class="f-i f-us">us</span></td><td class="left iz" data-stat="college"></td><td class="right" data-stat="year_min">2004</td><td class="right" data-stat="year_max">2019</td><td class="right" data-stat="g">1198</td><td class="right" data-stat="mp_per_g">38.6</td><td class="right" data-stat="pts_per_g">27.2</td><td class="right" data-stat="trb_per_g">7.4</td><td class="r

In [8]:
rows = table.find_all('tr')
rows[0].find_all('td')[0].get_text()

'2003'

In [9]:
lst1 = []
lst2 = []
for row in rows:
    td = row.find_all('td')
    lst3 = []
    for item in td:
        lst3.append(item.get_text())
        if row == rows[0]:
            lst2.append(item.get("data-stat"))
    lst1.append(lst3)
        

In [41]:
lst1

[['1985',
  'NBA',
  '1',
  '13',
  'UTA',
  'Karl Malone',
  '21.335',
  'F',
  'us',
  'Louisiana Tech',
  '1986',
  '2004',
  '1476',
  '37.2',
  '25.0',
  '10.1',
  '3.6',
  '1.4',
  '0.8',
  '.516',
  '.519',
  '.274',
  '.742',
  '234.6',
  '.205'],
 ['2003',
  'NBA',
  '1',
  '1',
  'CLE',
  'LeBron James',
  '18.182',
  'F-G',
  'us',
  '',
  '2004',
  '2019',
  '1198',
  '38.6',
  '27.2',
  '7.4',
  '7.2',
  '1.6',
  '0.8',
  '.504',
  '.548',
  '.343',
  '.736',
  '226.6',
  '.235'],
 ['1997',
  'NBA',
  '1',
  '1',
  'SAS',
  'Tim Duncan',
  '21.066',
  'F-C',
  'vi',
  'Wake Forest',
  '1998',
  '2016',
  '1392',
  '34.0',
  '19.0',
  '10.8',
  '3.0',
  '0.7',
  '2.2',
  '.506',
  '.509',
  '.179',
  '.696',
  '206.4',
  '.209'],
 ['1998',
  'NBA',
  '1',
  '9',
  'MIL',
  'Dirk Nowitzki',
  '20.010',
  'F',
  'de',
  '',
  '1999',
  '2019',
  '1522',
  '33.8',
  '20.7',
  '7.5',
  '2.4',
  '0.8',
  '0.8',
  '.471',
  '.496',
  '.380',
  '.879',
  '206.3',
  '.193'],
 ['199

In [11]:
lst2

['year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_country',
 'college',
 'year_min',
 'year_max',
 'g',
 'mp_per_g',
 'pts_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'fg_pct',
 'fg2_pct',
 'fg3_pct',
 'ft_pct',
 'ws',
 'ws_per_48']

In [42]:
lst1 = []
lst2 = []
count = 0
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2015&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=0'
for i in range(45):
    if i >0:
        href_list = soup.find("div", {"class": "p402_premium"}).find('p').find_all('a')
        if len(href_list) == 1:
            count += 1
            print(count)
            if count == 2:
                break
            href = href_list[0].get('href')
        else:
            href = href_list[1].get('href')
        print(href)
        draft_url= 'https://www.basketball-reference.com' + href
        time.sleep(4*random.random())
    r = requests.get(draft_url)
    print(r.status_code)
    soup = BeautifulSoup(r.content, "html")
    div = soup.find("div", {"class": "table_outer_container"})
    table = div.find("tbody")
    rows = table.find_all('tr')
    for row in rows:
        tds = row.find_all('td')
        lst3 = []
        for item in tds:
            lst3.append(item.get_text())
        lst1.append(lst3)
for i in rows[0].find_all('td'):
    lst2.append(i.get("data-stat"))
        

200
1
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2015&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=100
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2015&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=200
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2015&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_

In [36]:
div2 = soup.find("div", {"class": "p402_premium"})
div2

['year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_country',
 'college',
 'year_min',
 'year_max',
 'g',
 'mp_per_g',
 'pts_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'fg_pct',
 'fg2_pct',
 'fg3_pct',
 'ft_pct',
 'ws',
 'ws_per_48',
 'year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_country',
 'college',
 'year_min',
 'year_max',
 'g',
 'mp_per_g',
 'pts_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'fg_pct',
 'fg2_pct',
 'fg3_pct',
 'ft_pct',
 'ws',
 'ws_per_48',
 'year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_country',
 'college',
 'year_min',
 'year_max',
 'g',
 'mp_per_g',
 'pts_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'fg_pct',
 'fg2_pct',
 'fg3_pct',
 'ft_pct',
 'ws',
 'ws_per_48',
 'year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_coun

In [14]:
p = div2.find('p').find('a')
p.get('href')

'/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2015&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=2000'

In [43]:
print(len(lst1))
#pd.DataFrame(lst1[1], index=lst2)
lst2

2292


['year_id',
 'lg_id',
 'round',
 'pick_overall',
 'team_id',
 'player',
 'age',
 'pos',
 'birth_country',
 'college',
 'year_min',
 'year_max',
 'g',
 'mp_per_g',
 'pts_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'fg_pct',
 'fg2_pct',
 'fg3_pct',
 'ft_pct',
 'ws',
 'ws_per_48']

In [252]:
all_star_list = []
url = 'https://www.basketball-reference.com/allstar/NBA-allstar-career-stats.html'
r = requests.get(url)
soup1 = BeautifulSoup(r.content, "html")
div = soup1.find("div", {"class": "table_outer_container"})
table = div.find("tbody")
rows = table.find_all('tr')
for row in rows:
    a_tag =row.find('th').find('a')
    if a_tag == None:
        continue
    player = a_tag.get_text()
    href = a_tag.get('href')
    all_star_list.append(player)
data = pd.DataFrame()
for playr in lst1[0:2]:
    try:
        playr[5]
    except:
        continue
    player = playr[5]
    split_player = player.split()
    formatted_player = '-'.join(split_player)
    try:
        (formatted_player +'-1').lower().replace("'", "")
    except:
        print(playr)
    final_format = (formatted_player +'-1').lower().replace("'", "")
    #print(final_format)
    print(playr)
    try:
        Player(final_format)
    except:
        continue
    try:
        Player(final_format).dataframe.dropna(axis=1).iloc[-1,:]
    except:
        continue
    name = Player(final_format).dataframe.dropna(axis=1).iloc[-1,:]
    #print(name.values)
    #print(len(name.values))
    data = pd.concat([data,pd.DataFrame(name.values.reshape(1,len(name.values)),index=[player], columns=[name.index])])
    columns =name.index
    #https://www.sports-reference.com/cbb/players/kareem-abdul-jabbar-1.html
#pd.DataFrame(name.values,index=playr[5])
data

['1985', 'NBA', '1', '13', 'UTA', 'Karl Malone', '21.335', 'F', 'us', 'Louisiana Tech', '1986', '2004', '1476', '37.2', '25.0', '10.1', '3.6', '1.4', '0.8', '.516', '.519', '.274', '.742', '234.6', '.205']
['2003', 'NBA', '1', '1', 'CLE', 'LeBron James', '18.182', 'F-G', 'us', '', '2004', '2019', '1198', '38.6', '27.2', '7.4', '7.2', '1.6', '0.8', '.504', '.548', '.343', '.736', '226.6', '.235']


,assists,blocks,conference,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,...,player_id,points,position,steals,team_abbreviation,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,weight
Karl Malone,125,59,,1154,0.566,653,0.563,650,0.631,410,...,karl-malone-1,1716,Forward,151,louisiana-tech,859,0.587,15.5,269,250


In [90]:
Player(final_format)


In [97]:
nba = []
for player in data.index:
    if player in all_star_list:
        nba.append(1)
    else: 
        nba.append(0)

In [106]:
data.to_pickle('drafted_players_college.pkl')

In [102]:
data.drop(columns="All-Star",inplace = True)

In [104]:
boost = GradientBoostingClassifier()

In [255]:
data = pd.read_pickle('drafted_players_college.pkl')

In [256]:
data.columns

MultiIndex([(              'assist_percentage',),
            (                        'assists',),
            (               'block_percentage',),
            (                         'blocks',),
            (                 'box_plus_minus',),
            (                     'conference',),
            (       'defensive_box_plus_minus',),
            (   'defensive_rebound_percentage',),
            (             'defensive_rebounds',),
            (           'defensive_win_shares',),
            ('effective_field_goal_percentage',),
            (            'field_goal_attempts',),
            (          'field_goal_percentage',),
            (                    'field_goals',),
            (        'free_throw_attempt_rate',),
            (            'free_throw_attempts',),
            (          'free_throw_percentage',),
            (                    'free_throws',),
            (                   'games_played',),
            (                  'games_started',),


In [257]:
data.columns = [col[0] for col in data.columns]

In [258]:
def height_func(row):
    try: 
        row['height'].split('-')
    except:
        return 6.56
    split = row['height'].split('-')
    return int(split[0])+(int(split[1])/12)

In [259]:
data['height'] = data.apply(height_func ,axis=1)

In [260]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1404 entries, Karl Malone to Cady Lalanne
Data columns (total 51 columns):
assist_percentage                  327 non-null object
assists                            1376 non-null object
block_percentage                   327 non-null object
blocks                             1379 non-null object
box_plus_minus                     135 non-null object
conference                         1404 non-null object
defensive_box_plus_minus           135 non-null object
defensive_rebound_percentage       175 non-null object
defensive_rebounds                 561 non-null object
defensive_win_shares               752 non-null object
effective_field_goal_percentage    1126 non-null object
field_goal_attempts                1392 non-null object
field_goal_percentage              1389 non-null object
field_goals                        1392 non-null object
free_throw_attempt_rate            1389 non-null object
free_throw_attempts                1392 non-nul

In [262]:
data2 = data.copy(deep=True)

In [264]:
data2.dtypes

assist_percentage                   object
assists                             object
block_percentage                    object
blocks                              object
box_plus_minus                      object
conference                          object
defensive_box_plus_minus            object
defensive_rebound_percentage        object
defensive_rebounds                  object
defensive_win_shares                object
effective_field_goal_percentage     object
field_goal_attempts                 object
field_goal_percentage               object
field_goals                         object
free_throw_attempt_rate             object
free_throw_attempts                 object
free_throw_percentage               object
free_throws                         object
games_played                        object
games_started                       object
height                             float64
minutes_played                      object
offensive_box_plus_minus            object
offensive_r

In [270]:
data2.drop(columns=['player_id','team_abbreviation'],inplace = True)

In [272]:
data2.drop(columns=['position'],inplace = True)

In [273]:
for column in data2.columns:
    print(column)
    data2[column].astype(float)
    data2[column].fillna((data2[column].mean()), inplace=True) 

assist_percentage
assists
block_percentage
blocks
box_plus_minus
defensive_box_plus_minus
defensive_rebound_percentage
defensive_rebounds
defensive_win_shares
effective_field_goal_percentage
field_goal_attempts
field_goal_percentage
field_goals
free_throw_attempt_rate
free_throw_attempts
free_throw_percentage
free_throws
games_played
games_started
height
minutes_played
offensive_box_plus_minus
offensive_rebound_percentage
offensive_rebounds
offensive_win_shares
personal_fouls
player_efficiency_rating
points
points_produced
steal_percentage
steals
three_point_attempt_rate
three_point_attempts
three_point_percentage
three_pointers
total_rebound_percentage
total_rebounds
true_shooting_percentage
turnover_percentage
turnovers
two_point_attempts
two_point_percentage
two_pointers
usage_percentage
weight
win_shares
win_shares_per_40_minutes


In [277]:
boost = GradientBoostingClassifier()

In [278]:
boost.fit(data2, nba)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [282]:
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=2016&year_max=2018&college_id=0&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&order_by=ws'
r = requests.get(draft_url)
soup = BeautifulSoup(r.content, "html")
after_2015 = []
count = 0
for i in range(20):
    if i >0:
        href_list = soup.find("div", {"class": "p402_premium"}).find('p').find_all('a')
        if len(href_list) == 1:
            count +=1
            print(count)
            if count == 2:
                break
            href = href_list[0].get('href')
        else:
            href = href_list[1].get('href')
        print(href)
        draft_url= 'https://www.basketball-reference.com' + href
        time.sleep(4*random.random())
    r = requests.get(draft_url)
    print(r.status_code)
    soup = BeautifulSoup(r.content, "html")
    div = soup.find("div", {"class": "table_outer_container"})
    table = div.find("tbody")
    rows = table.find_all('tr')
    for row in rows:
        tds = row.find_all('td')
        lst3 = []
        for item in tds:
            lst3.append(item.get_text())
        after_2015.append(lst3)

200
1
/play-index/draft_finder.cgi?request=1&year_min=2016&year_max=2018&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=100
200
2


In [284]:
over2015 = pd.DataFrame()
for playr in after_2015[0:2]:
    try:
        playr[5]
    except:
        continue
    player = playr[5]
    split_player = player.split()
    formatted_player = '-'.join(split_player)
    try:
        (formatted_player +'-1').lower().replace("'", "")
    except:
        print(playr)
    final_format = (formatted_player +'-1').lower().replace("'", "")
    #print(final_format)
    print(playr)
    try:
        Player(final_format)
    except:
        continue
    try:
        Player(final_format).dataframe.dropna(axis=1).iloc[-1,:]
    except:
        continue
    name = Player(final_format).dataframe.dropna(axis=1).iloc[-1,:]
    #print(name.values)
    #print(len(name.values))
    data_after_2015 = pd.concat([data,pd.DataFrame(name.values.reshape(1,len(name.values)),index=[player], columns=[name.index])])
    #https://www.sports-reference.com/cbb/players/kareem-abdul-jabbar-1.html
#pd.DataFrame(name.values,index=playr[5])
data_after_2015

['2016', 'NBA', '1', '1', 'PHI', 'Ben Simmons', '19.343', 'G-F', 'au', 'LSU', '2018', '2019', '160', '34.0', '16.4', '8.5', '7.9', '1.6', '0.8', '.554', '.558', '.000', '.583', '17.5', '.154']


TypeError: Expected tuple, got str